#### Evaluating the generated documentation

In [24]:
import pandas as pd
import numpy as np
import json
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu
from evaluate import load

In [4]:
DATA_PATH = 'C:\\Users\\dmasrour\\Documents\\CodeDoc_Generation\\Data_backup\\DocGen_Unprocessed_Notebooks\\kaggle-courses-master\\intro_to_machine_learning\\'

In [14]:
with open(DATA_PATH + '03-your-first-machine-learning-model.ipynb', encoding="utf8") as jsonfile:
    original_nb = json.load(jsonfile)
original_nb = pd.json_normalize(original_nb['cells'])[['cell_type', 'source']]

In [15]:
with open(DATA_PATH + '03-your-first-machine-learning-model.ipynbPLBART_documented.ipynb', encoding="utf8") as jsonfile:
    docgen_nb = json.load(jsonfile)
docgen_nb = pd.json_normalize(docgen_nb['cells'])[['cell_type', 'source']]

In [17]:
original_nb.head(), original_nb.shape

(  cell_type                                             source
 0  markdown  [**[Introduction to Machine Learning Home Page...
 1  markdown  [## Recap\n, So far, you have loaded your data...
 2      code  [# Code you have previously used to load data\...
 3  markdown  [# Exercises\n, \n, ## Step 1: Specify Predict...
 4      code  [# print the list of columns in the dataset to...,
 (22, 2))

In [18]:
docgen_nb.head(), docgen_nb.shape

(  cell_type                                             source
 0  markdown      [Loads the data from the previous used code.]
 1      code  [# Code you have previously used to load data\...
 2  markdown  [Print the list of columns in the dataset to t...
 3      code  [# print the list of columns in the dataset to...
 4  markdown                  [Check the value of thealePrice.],
 (16, 2))

#### BERT score (takes too long without GPU)

In [22]:
bertscore = load("bertscore")

In [ ]:
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

#### Smoothed BLEU-1 score

In [50]:
def bleu(ref, gen):
    ''' 
    calculate pair wise bleu score. uses nltk implementation
    Args:
        references : a list of reference sentences 
        candidates : a list of candidate(generated) sentences
    Returns:
        bleu score(float)
    '''
    ref_bleu = []
    gen_bleu = []
    for l in gen:
        gen_bleu.append(l.split())
    for i,l in enumerate(ref):
        ref_bleu.append([l.split()])
    cc = SmoothingFunction()
    score_bleu = corpus_bleu(ref_bleu, gen_bleu, weights=(1, 0, 0, 0), smoothing_function=cc.method4)
    return score_bleu

In [51]:
predictions = ['Create the list of features below the given feature names.']
references = ["After you've created that list of features, use it to create the DataFrame that you'll use to fit the model."]
bleu(references, predictions)

0.14715177646857694

#### BERT vectors cosine similarity

In [54]:
from transformers import AutoTokenizer, AutoModel, pipeline
import torch
from scipy.spatial.distance import cosine

In [53]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
model = AutoModel.from_pretrained("microsoft/codebert-base")

In [59]:
ref = "Select the target variable, which corresponds to the sales price. Save this to a new variable called `y`. You'll need to print a list of the columns to find the name of the column you need."
toks_ref = tokenizer.tokenize(ref)

tokens = [tokenizer.cls_token] + toks_ref + [tokenizer.sep_token]

tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
ref_embeddings = model(torch.tensor(tokens_ids)[None,:])[0]

In [63]:
gen = 'hello'
toks_gen = tokenizer.tokenize(gen)

tokens = [tokenizer.cls_token] + toks_gen + [tokenizer.sep_token]

tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
gen_embeddings = model(torch.tensor(tokens_ids)[None,:])[0]

In [64]:
# convert tensor into np array
tensor_np_ref = ref_embeddings.cpu().detach().numpy()
tensor_np_gen = gen_embeddings.cpu().detach().numpy()
# average of embeddings of the tokens in the sequence
avg_ref = np.mean(tensor_np_ref[0], axis=0)
avg_gen = np.mean(tensor_np_gen[0], axis=0)

In [65]:
cos = 1 - cosine(avg_ref, avg_gen)
cos

0.8215266466140747